## Compare untuned model vs. tuned model

### Imports and load data from csv

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa

from keras.layers import Dense, Conv1D, Flatten, GlobalAveragePooling1D, TimeDistributed, LSTM, AveragePooling1D, SimpleRNN, LeakyReLU

D:\Software\Anaconda\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
import numpy.random as rd

In [3]:
comb_raw = pd.read_csv("SeqCombinedData.csv")
display(comb_raw)

,Usage,LaggingCurrentReactivePower,LeadingCurrentReactivePower,CO2,LaggingCurrentPowerFactor,LeadingCurrentPowerFactor,NSM,WeekStatus,Year,Month,...,SeaLevelPressure,CloudCover,Visibility,SolarRadiation,SolarEnergy,UvIndex,Conditions,SunriseHour,LoadType,SunsetHour
0,3.17,2.95,0.00,0.0,73.21,100.00,900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
1,3.28,3.56,0.00,0.0,67.76,100.00,5400,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
2,3.46,4.03,0.00,0.0,65.14,100.00,9900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
3,3.89,5.00,0.00,0.0,61.40,100.00,14400,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
4,3.56,4.07,0.00,0.0,65.84,100.00,18900,Weekday,2018,1,...,1026.9,3.3,9.0,139.9,12.0,6,Clear,7,Light_Load,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,3.42,0.00,9.79,0.0,100.00,32.98,64800,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7004,3.96,0.00,18.29,0.0,100.00,21.16,69300,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7005,3.38,0.00,13.43,0.0,100.00,24.41,73800,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17
7006,3.42,0.00,13.36,0.0,100.00,24.80,78300,Weekday,2018,12,...,1035.8,19.6,10.5,143.6,12.5,6,Clear,7,Light_Load,17


In [4]:
comb_raw_filt= comb_raw[["Usage", "LaggingCurrentReactivePower", "LeadingCurrentReactivePower",
                          "CO2", "LaggingCurrentPowerFactor", "LeadingCurrentPowerFactor", "NSM",
                          "WeekStatus", "Hours", "Minutes", "IsHoliday", "Season", "Temp", "Dew",
                          "Humidity", "Precip", "PrecipProb", "PrecipCover", "PrecipType", "SnowDepth", "WindGust",
                          "WindSpeed", "WindDir", "SeaLevelPressure", "CloudCover", "Visibility", "SolarRadiation",
                          "SolarEnergy", "UvIndex", "Conditions", "SunriseHour", "LoadType", "SunsetHour"]]
load_values = comb_raw.pop("LoadType")

In [5]:
steel_cols_to_encode = ['IsHoliday','PrecipType','Season','Conditions','WeekStatus']
steel_cols_to_normalize = ['Usage', 'CO2', 'LaggingCurrentPowerFactor', 'PrecipCover', 'SnowDepth',
                          'NSM', 'Hours', 'Minutes', 'PrecipProb', 'SunriseHour']

In [6]:
comb_raw_enc = pd.get_dummies(comb_raw_filt, columns=steel_cols_to_encode)

In [7]:
steel_scaler = StandardScaler()
comb_raw_enc[steel_cols_to_normalize] = steel_scaler.fit_transform(comb_raw_enc[steel_cols_to_normalize])

In [8]:
feature_selection_columns = [
    'Usage', 'LeadingCurrentReactivePower',
       'LaggingCurrentPowerFactor', 'LeadingCurrentPowerFactor', 'NSM',
       'Hours', 'WindDir', 'SunriseHour', 'SunsetHour', 'IsHoliday_0',
       'IsHoliday_1', 'PrecipType_rain,snow', 'PrecipType_snow',
       'Season_Autumn', 'Season_Spring', 'Conditions_Rain',
       'Conditions_Snow, Partially cloudy', 'Conditions_Snow, Rain',
       'Conditions_Snow, Rain, Overcast', 'WeekStatus_Weekday'
        ]
comb_raw_filt = comb_raw_enc[feature_selection_columns]
display(comb_raw_filt)

,Usage,LeadingCurrentReactivePower,LaggingCurrentPowerFactor,LeadingCurrentPowerFactor,NSM,Hours,WindDir,SunriseHour,SunsetHour,IsHoliday_0,IsHoliday_1,"PrecipType_rain,snow",PrecipType_snow,Season_Autumn,Season_Spring,Conditions_Rain,"Conditions_Snow, Partially cloudy","Conditions_Snow, Rain","Conditions_Snow, Rain, Overcast",WeekStatus_Weekday
0,-0.722530,0.00,-0.388131,100.00,-1.678015,-1.661325,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
1,-0.719255,0.00,-0.675669,100.00,-1.497584,-1.516862,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
2,-0.713895,0.00,-0.813898,100.00,-1.317152,-1.372399,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
3,-0.701091,0.00,-1.011218,100.00,-1.136720,-1.083473,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
4,-0.710917,0.00,-0.776967,100.00,-0.956288,-0.939010,335.8,1.318323,17,0,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7003,-0.715086,9.79,1.025289,32.98,0.884116,0.939010,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7004,-0.699006,18.29,1.025289,21.16,1.064547,1.083473,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7005,-0.716277,13.43,1.025289,24.41,1.244979,1.227936,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1
7006,-0.715086,13.36,1.025289,24.80,1.425411,1.372399,335.6,1.318323,17,1,0,0,0,0,0,0,0,0,0,1


In [9]:
transformed_comb = []
for num in range(4,len(comb_raw_filt)):
    ele =comb_raw_filt[num-4:num+1]
    transformed_comb.append(ele)
transformed_comb = np.array(transformed_comb)

In [10]:
X_arr = transformed_comb
Y_arr = load_values[4:]
print(len(Y_arr))

7004


### Metrics of baseline over 5 trials

In [11]:
random_seeds = [10, 15, 20, 25, 30]
untuned_acc_array = []
untuned_precision_array=[]
untuned_f1_array=[]

for itr in random_seeds:
    X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size=0.20, random_state=itr)
    y_train_le = pd.get_dummies(y_train)
    y_test_le = pd.get_dummies(y_test)
    
    model_1 = tf.keras.models.Sequential()
    model_1.add(LSTM(20, return_sequences=True))
    model_1.add(LSTM(20, return_sequences=False))
    model_1.add(Dense(3, activation='softmax'))

    model_1.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy',
                      tf.keras.metrics.Precision(),
                      tfa.metrics.F1Score(num_classes=3,
                                                  average='macro',
                                                  threshold=0.5)])
    
    model_1.fit(x=X_train, y=y_train_le, epochs=20)
    untuned_baseline_score = model_1.evaluate(X_test, y_test_le, verbose=0)
    untuned_acc_array.append(untuned_baseline_score[1])
    untuned_precision_array.append(untuned_baseline_score[2])
    untuned_f1_array.append(untuned_baseline_score[3])

Epoch 1/20
176/176 [==============================] - 10s 10ms/step - loss: 0.9887 - accuracy: 0.5145 - precision: 0.6061 - f1_score: 0.1940
Epoch 2/20
176/176 [==============================] - 2s 10ms/step - loss: 0.7451 - accuracy: 0.6705 - precision: 0.8163 - f1_score: 0.4087
Epoch 3/20
176/176 [==============================] - 2s 10ms/step - loss: 0.5807 - accuracy: 0.7430 - precision: 0.8206 - f1_score: 0.5599
Epoch 4/20
176/176 [==============================] - 2s 9ms/step - loss: 0.4900 - accuracy: 0.7896 - precision: 0.8267 - f1_score: 0.7018
Epoch 5/20
176/176 [==============================] - 2s 9ms/step - loss: 0.4387 - accuracy: 0.8072 - precision: 0.8390 - f1_score: 0.7429
Epoch 6/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4121 - accuracy: 0.8224 - precision: 0.8451 - f1_score: 0.7611
Epoch 7/20
176/176 [==============================] - 2s 10ms/step - loss: 0.3848 - accuracy: 0.8363 - precision: 0.8515 - f1_score: 0.7836
Epoch 8/20
176/176 [=

In [12]:
print(untuned_acc_array)
print(untuned_precision_array)
print(untuned_f1_array)

[0.8779443502426147, 0.8772305250167847, 0.9107779860496521, 0.8893647193908691, 0.8865096569061279]
[0.8833333253860474, 0.8833333253860474, 0.9242534637451172, 0.897230327129364, 0.8918722867965698]
[0.8353815078735352, 0.8367884159088135, 0.8920130729675293, 0.8577966094017029, 0.8517574667930603]


### Running GWO on a model for hyperparameters and evaluating the results

In [13]:
def gwo_nn(model,train_feature,train_label,SearchAgents_no,Max_iteration,dim):
    loop_model = tf.keras.models.clone_model(model)
    Alpha_pos=[0,0,0]
    Beta_pos=[0,0,0]
    Delta_pos=[0,0,0]
    
    lower = [0.0009, 0.5, 0.800]
    upper = [0.05, 1.5, 1]
    bounds = np.subtract(upper,lower)

    Alpha_score = float("inf")       
    Beta_score = float("inf")
    Delta_score = float("inf")
 
    #Positions = np.dot(rd.rand(SearchAgents_no,dim),(ub-lb))+lb
    Positions = (rd.rand(SearchAgents_no,dim) * bounds) + lower
    
    Convergence_curve=np.zeros((1,Max_iteration))

    iterations = []
    accuracy = []

    index_iteration = 0 
    while index_iteration < Max_iteration:
        for i in range(0,(Positions.shape[0])):
            for j in range(0,(Positions.shape[1])):
                Flag4ub=Positions[i,j]>upper[j]
                Flag4lb=Positions[i,j]<lower[j]
                if Flag4ub:                   
                    Positions[i,j] = upper[j]
                if Flag4lb:                   
                    Positions[i,j] = lower[j]
            
            adam_optimizer = tf.keras.optimizers.Adam(learning_rate = Positions[i][0], beta_1=Positions[i][1], beta_2=Positions[i][2])
            loop_model = tf.keras.models.clone_model(model)
            loop_model.compile(loss='categorical_crossentropy',optimizer=adam_optimizer,metrics=['accuracy'])
            loop_model.fit(x=train_feature, y=train_label, epochs=20)
            acc = loop_model.history.history['accuracy']
            #print(acc)
            scores=acc[len(acc)-1]
            
            fitness = (1 - scores)*100
            if fitness<Alpha_score:
                Alpha_score=fitness
                Alpha_pos=Positions[i]
            if fitness>Alpha_score and fitness<Beta_score:
                Beta_score=fitness
                Beta_pos=Positions[i]
            if fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score:
                Delta_score=fitness
                Delta_pos=Positions[i]


        a=2-index_iteration*(2/Max_iteration)
        
        for i in range(0,(Positions.shape[0])):
            for j in range(0,(Positions.shape[1])):              
                r1=rd.random(1)
                r2=rd.random(1)               
                A1=2*a*r1-a
                C1=2*r2

                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j])
                X1=Alpha_pos[j]-A1*D_alpha
                       
                r1=rd.random(1)
                r2=rd.random(1)

                A2=2*a*r1-a
                C2=2*r2

                D_beta=abs(C2*Beta_pos[j]-Positions[i,j])
                X2=Beta_pos[j]-A2*D_beta

                r1=rd.random(1)
                r2=rd.random(1)

                A3=2*a*r1-a
                C3=2*r2

                D_delta=abs(C3*Delta_pos[j]-Positions[i,j])
                X3=Delta_pos[j]-A3*D_delta

                Positions[i,j]=(X1+X2+X3)/3

        
        index_iteration = index_iteration + 1
        iterations.append(index_iteration)
        accuracy.append((100-Alpha_score)/100)
        print('---------------------------Completeting itr--------------------' + str(index_iteration))
        print(Positions)
        print('Learning rate, B1, B2:' + str(Alpha_pos))
        print('accuracy:' + str((100-Alpha_score)/100))

    bestLR=Alpha_pos[0]
    bestB1=Alpha_pos[1]
    bestB2=Alpha_pos[2]

    return bestLR,bestB1,bestB2,iterations,accuracy

### Metrics for GWO tuned model over 5 trials

In [14]:
random_seeds = [10, 15, 20, 25, 30]
tuned_acc_array = []
tuned_precision_array=[]
tuned_f1_array=[]

for itr in random_seeds:
    X_train, X_test, y_train, y_test = train_test_split(X_arr, Y_arr, test_size=0.20, random_state=itr)
    y_train_le = pd.get_dummies(y_train)
    y_test_le = pd.get_dummies(y_test)
    
    model_2 = tf.keras.models.Sequential()
    model_2.add(LSTM(20, return_sequences=True))
    model_2.add(LSTM(20, return_sequences=False))
    model_2.add(Dense(3, activation='softmax'))

    model_2.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy',
                      tf.keras.metrics.Precision(),
                      tfa.metrics.F1Score(num_classes=3,
                                                  average='macro',
                                                  threshold=0.5)])
    
    SearchAgents_no=10
    Max_iteration= 5
    dim=3
    bestLR,bestB1,bestB2,iterations,accuracy = gwo_nn(model_2,X_train,y_train_le,SearchAgents_no,Max_iteration,dim)
    
    model_3 = tf.keras.models.Sequential()
    model_3.add(LSTM(20, return_sequences=True))
    model_3.add(LSTM(20, return_sequences=False))
    model_3.add(Dense(3, activation='softmax'))

    adam_optimizer = tf.keras.optimizers.Adam(learning_rate = bestLR, beta_1=bestB1, beta_2=bestB2)

    model_3.compile(loss='categorical_crossentropy',
              optimizer=adam_optimizer,
              metrics=['accuracy',
                      tf.keras.metrics.Precision(),
                      tfa.metrics.F1Score(num_classes=3,
                                                  average='macro',
                                                  threshold=0.5)])
    
    model_3.fit(x=X_train, y=y_train_le, epochs=20)
    tuned_baseline_score = model_3.evaluate(X_test, y_test_le, verbose=0)
    tuned_acc_array.append(tuned_baseline_score[1])
    tuned_precision_array.append(tuned_baseline_score[2])
    tuned_f1_array.append(tuned_baseline_score[3])

Epoch 1/20
176/176 [==============================] - 8s 10ms/step - loss: 1.0594 - accuracy: 0.5070
Epoch 2/20
176/176 [==============================] - 2s 10ms/step - loss: 1.0652 - accuracy: 0.5101
Epoch 3/20
176/176 [==============================] - 2s 10ms/step - loss: 1.1138 - accuracy: 0.5103
Epoch 4/20
176/176 [==============================] - 2s 10ms/step - loss: 1.1808 - accuracy: 0.5104
Epoch 5/20
176/176 [==============================] - 2s 10ms/step - loss: 1.2517 - accuracy: 0.5110
Epoch 6/20
176/176 [==============================] - 2s 10ms/step - loss: 1.3265 - accuracy: 0.5112
Epoch 7/20
176/176 [==============================] - 2s 10ms/step - loss: 1.3969 - accuracy: 0.5112
Epoch 8/20
176/176 [==============================] - 2s 9ms/step - loss: nan - accuracy: 0.5099
Epoch 9/20
176/176 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.5099
Epoch 10/20
176/176 [==============================] - 2s 10ms/step - loss: nan - accuracy: 0.5099

176/176 [==============================] - 2s 10ms/step - loss: 0.4555 - accuracy: 0.7944
Epoch 4/20
176/176 [==============================] - 2s 10ms/step - loss: 0.5030 - accuracy: 0.7714
Epoch 5/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4622 - accuracy: 0.7976
Epoch 6/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4249 - accuracy: 0.8062
Epoch 7/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4354 - accuracy: 0.8040
Epoch 8/20
176/176 [==============================] - 2s 11ms/step - loss: 0.4201 - accuracy: 0.8062
Epoch 9/20
176/176 [==============================] - 2s 9ms/step - loss: 0.4244 - accuracy: 0.8171
Epoch 10/20
176/176 [==============================] - 2s 9ms/step - loss: 0.4176 - accuracy: 0.8165
Epoch 11/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4064 - accuracy: 0.8165
Epoch 12/20
176/176 [==============================] - 2s 10ms/step - loss: 0.4033 - accuracy: 0.8133


176/176 [==============================] - 2s 12ms/step - loss: 1.0301 - accuracy: 0.5099
Epoch 19/20
176/176 [==============================] - 2s 12ms/step - loss: 1.0286 - accuracy: 0.5099
Epoch 20/20
176/176 [==============================] - 2s 11ms/step - loss: 1.0281 - accuracy: 0.5099
Epoch 1/20
176/176 [==============================] - 10s 13ms/step - loss: 0.7337 - accuracy: 0.6638
Epoch 2/20
176/176 [==============================] - 2s 12ms/step - loss: 0.4498 - accuracy: 0.8047
Epoch 3/20
176/176 [==============================] - 2s 13ms/step - loss: 0.4650 - accuracy: 0.7930
Epoch 4/20
176/176 [==============================] - 2s 11ms/step - loss: 0.4101 - accuracy: 0.8169
Epoch 5/20
176/176 [==============================] - 2s 12ms/step - loss: 0.4039 - accuracy: 0.8142
Epoch 6/20
176/176 [==============================] - 2s 13ms/step - loss: 0.3965 - accuracy: 0.8221
Epoch 7/20
176/176 [==============================] - 2s 13ms/step - loss: 0.3871 - accuracy: 0.822

176/176 [==============================] - 2s 13ms/step - loss: 0.2709 - accuracy: 0.8892
Epoch 15/20
176/176 [==============================] - 2s 12ms/step - loss: 0.2598 - accuracy: 0.8961
Epoch 16/20
176/176 [==============================] - 2s 12ms/step - loss: 0.2659 - accuracy: 0.8899
Epoch 17/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2385 - accuracy: 0.9018
Epoch 18/20
176/176 [==============================] - 2s 12ms/step - loss: 0.2432 - accuracy: 0.9036
Epoch 19/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2512 - accuracy: 0.9004
Epoch 20/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2319 - accuracy: 0.9065
Epoch 1/20
176/176 [==============================] - 9s 13ms/step - loss: 0.8533 - accuracy: 0.5940
Epoch 2/20
176/176 [==============================] - 2s 13ms/step - loss: 0.4852 - accuracy: 0.7857
Epoch 3/20
176/176 [==============================] - 2s 13ms/step - loss: 0.4142 - accuracy: 0.

176/176 [==============================] - 2s 13ms/step - loss: 0.2944 - accuracy: 0.8674
Epoch 9/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2800 - accuracy: 0.8785
Epoch 10/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2695 - accuracy: 0.8835
Epoch 11/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2509 - accuracy: 0.8951
Epoch 12/20
176/176 [==============================] - 2s 13ms/step - loss: 0.2444 - accuracy: 0.8970
Epoch 13/20
176/176 [==============================] - 2s 12ms/step - loss: 0.2118 - accuracy: 0.9156
Epoch 14/20
176/176 [==============================] - 2s 13ms/step - loss: 0.1972 - accuracy: 0.9202
Epoch 15/20
176/176 [==============================] - 2s 13ms/step - loss: 0.1822 - accuracy: 0.9284
Epoch 16/20
176/176 [==============================] - 2s 13ms/step - loss: 0.1704 - accuracy: 0.9363
Epoch 17/20
176/176 [==============================] - 2s 13ms/step - loss: 0.1662 - accuracy: 

...

176/176 [==============================] - 1s 4ms/step - loss: 0.5250 - accuracy: 0.7826
Epoch 13/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4239 - accuracy: 0.8094
Epoch 14/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4131 - accuracy: 0.8135
Epoch 15/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4173 - accuracy: 0.8112
Epoch 16/20
176/176 [==============================] - 1s 4ms/step - loss: 0.6044 - accuracy: 0.7776
Epoch 17/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4246 - accuracy: 0.8067
Epoch 18/20
176/176 [==============================] - 1s 4ms/step - loss: 0.6631 - accuracy: 0.7467
Epoch 19/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4508 - accuracy: 0.8035
Epoch 20/20
176/176 [==============================] - 1s 4ms/step - loss: 0.4909 - accuracy: 0.7892
Epoch 1/20
176/176 [==============================] - 3s 4ms/step - loss: 1.2372 - accuracy: 0.2115
Epo

176/176 [==============================] - 1s 4ms/step - loss: 0.3459 - accuracy: 0.8522
Epoch 9/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3368 - accuracy: 0.8501
Epoch 10/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3240 - accuracy: 0.8599
Epoch 11/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3183 - accuracy: 0.8567
Epoch 12/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3095 - accuracy: 0.8604
Epoch 13/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3047 - accuracy: 0.8667
Epoch 14/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2961 - accuracy: 0.8706
Epoch 15/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2891 - accuracy: 0.8719
Epoch 16/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2852 - accuracy: 0.8720
Epoch 17/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2754 - accuracy: 0.8799
Epo

176/176 [==============================] - 1s 4ms/step - loss: 0.2609 - accuracy: 0.8842
Epoch 11/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2396 - accuracy: 0.8979
Epoch 12/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2294 - accuracy: 0.9013
Epoch 13/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2144 - accuracy: 0.9111
Epoch 14/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1964 - accuracy: 0.9172
Epoch 15/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1837 - accuracy: 0.9222
Epoch 16/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1926 - accuracy: 0.9186
Epoch 17/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1699 - accuracy: 0.9331
Epoch 18/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1681 - accuracy: 0.9334
Epoch 19/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1594 - accuracy: 0.9329
Ep

176/176 [==============================] - 1s 4ms/step - loss: 0.3359 - accuracy: 0.8538
Epoch 8/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3286 - accuracy: 0.8535
Epoch 9/20
176/176 [==============================] - 1s 4ms/step - loss: 0.3076 - accuracy: 0.8660
Epoch 10/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2847 - accuracy: 0.8752
Epoch 11/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2936 - accuracy: 0.8697
Epoch 12/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2659 - accuracy: 0.8822
Epoch 13/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2671 - accuracy: 0.8820
Epoch 14/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2627 - accuracy: 0.8849
Epoch 15/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2436 - accuracy: 0.8954
Epoch 16/20
176/176 [==============================] - 1s 4ms/step - loss: 0.2391 - accuracy: 0.8979
Epoc

176/176 [==============================] - 1s 4ms/step - loss: 0.1962 - accuracy: 0.9183 - precision_14: 0.9248 - f1_score: 0.8971
Epoch 18/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1851 - accuracy: 0.9241 - precision_14: 0.9305 - f1_score: 0.9033
Epoch 19/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1830 - accuracy: 0.9231 - precision_14: 0.9284 - f1_score: 0.9053
Epoch 20/20
176/176 [==============================] - 1s 4ms/step - loss: 0.1669 - accuracy: 0.9299 - precision_14: 0.9354 - f1_score: 0.9121


In [15]:
print(tuned_acc_array)
print(tuned_precision_array)
print(tuned_f1_array)

[0.9221984148025513, 0.8515346050262451, 0.9122055768966675, 0.9271948337554932, 0.8943611979484558]
[0.9234620928764343, 0.8582791090011597, 0.9157060384750366, 0.9308855533599854, 0.8954903483390808]
[0.8993996977806091, 0.7985641360282898, 0.887539803981781, 0.9121489524841309, 0.8758509755134583]


### Z-scores

In [17]:
import scipy.stats as stats

In [ ]:
print(stats.zscore(untuned_acc_array))

In [ ]:
print(stats.zscore(tuned_acc_array))

In [27]:
#fvalue_f1, pvalue_f1 = stats.f_oneway(untuned_acc_array, tuned_acc_array)
#print(fvalue_f1, pvalue_f1)

0.7683027290091241 0.40629230866540933


In [28]:
print(stats.zscore(untuned_precision_array))

[-0.8400202  -0.8400202   1.87272112  0.08126141 -0.27394212]


In [29]:
print(stats.zscore(tuned_precision_array))

[ 0.71717028 -1.78302427  0.41967477  1.00190863 -0.35572941]


In [30]:
#fvalue_f1, pvalue_f1 = stats.f_oneway(untuned_precision_array, tuned_precision_array)
#print(fvalue_f1, pvalue_f1)

0.3383386452459077 0.5768082635860385


In [19]:
print(stats.zscore(untuned_f1_array))

[-0.94394178 -0.87536564  1.81641957  0.14862523 -0.14573739]


In [20]:
print(stats.zscore(tuned_f1_array))

[ 0.61842795 -1.90635317  0.32147284  0.93765142  0.02880096]


In [18]:
#fvalue_f1, pvalue_f1 = stats.f_oneway(untuned_f1_array, tuned_f1_array)
#print(fvalue_f1, pvalue_f1)

0.7899576823363451 0.4000475259490563


#### T-Test for Accuracy

In [26]:
t_stat, p_value = stats.ttest_ind(untuned_acc_array, tuned_acc_array)
print("T statistic:", t_stat)
print("P-value:", p_value)

T statistic: -0.8765287953108665
P-value: 0.4062923086654109


#### T-Test for Precision

In [23]:
t_stat, p_value = stats.ttest_ind(untuned_precision_array, tuned_precision_array)
print("T statistic:", t_stat)
print("P-value:", p_value)

T statistic: -0.58166884500195
P-value: 0.576808263586042


#### T-Test for F1 score

In [22]:
t_stat, p_value = stats.ttest_ind(untuned_f1_array, tuned_f1_array)
print("T statistic:", t_stat)
print("P-value:", p_value)

T statistic: -0.8887956358670697
P-value: 0.4000475259490551


### Difference in means

In [31]:
print(np.mean(untuned_acc_array))
print(np.mean(tuned_acc_array))

0.8883654475212097
0.9014989256858825


In [32]:
print(np.mean(untuned_precision_array))
print(np.mean(tuned_precision_array))

0.8960045456886292
0.9047646284103393


In [33]:
print(np.mean(untuned_f1_array))
print(np.mean(tuned_f1_array))

0.8547474145889282
0.8747007131576539


### Graphs

In [35]:
import plotly.express as px

In [38]:
acc_plot_df = pd.DataFrame()
acc_plot_df["TrialNo"] = [1,2,3,4,5]
acc_plot_df["UntunedAcc"] = untuned_acc_array
acc_plot_df["TunedAcc"] = tuned_acc_array

fig_1 = px.line(acc_plot_df, x=acc_plot_df["TrialNo"], y=acc_plot_df[["UntunedAcc","TunedAcc"]].columns, title='Plotting accuracy over the trials: untuned vs tuned models',
               labels = {'x': 'Trial No.', 'value': 'Accuracy'})
fig_1.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
    )
)

fig_1.show()

In [39]:
prec_plot_df = pd.DataFrame()
prec_plot_df["TrialNo"] = [1,2,3,4,5]
prec_plot_df["UntunedPrec"] = untuned_precision_array
prec_plot_df["TunedPrec"] = tuned_precision_array

fig_2 = px.line(prec_plot_df, x=prec_plot_df["TrialNo"], y=prec_plot_df[["UntunedPrec","TunedPrec"]].columns, title='Plotting precision over the trials: untuned vs tuned models',
               labels = {'x': 'Trial No.', 'value': 'Precision'})
fig_2.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
    )
)

fig_2.show()

In [40]:
f1_plot_df = pd.DataFrame()
f1_plot_df["TrialNo"] = [1,2,3,4,5]
f1_plot_df["UntunedF1"] = untuned_f1_array
f1_plot_df["TunedF1"] = tuned_f1_array

fig_2 = px.line(f1_plot_df, x=f1_plot_df["TrialNo"], y=f1_plot_df[["UntunedF1","TunedF1"]].columns, title='Plotting F1 over the trials: untuned vs tuned models',
               labels = {'x': 'Trial No.', 'value': 'F1'})
fig_2.update_layout(
    xaxis = dict(
        tickmode = 'linear',
        tick0 = 1,
        dtick = 1
    )
)

fig_2.show()

### Saving to an excel for later use

In [41]:
results_df = pd.DataFrame()
results_df["TrialNo"] = [1,2,3,4,5]
results_df["UntunedAcc"] = untuned_acc_array
results_df["TunedAcc"] = tuned_acc_array
results_df["UntunedPrec"] = untuned_precision_array
results_df["TunedPrec"] = tuned_precision_array
results_df["UntunedF1"] = untuned_f1_array
results_df["TunedF1"] = tuned_f1_array

In [43]:
results_df.to_csv('Untuned-Tuned-Results.csv', index=False)